# Getting Started with the P-Cluster 
```{attention}
Please make sure to find some time to go through the below material before the hackweek.

__Upon completion you__:
<div>
  <input type="checkbox" name="a1">
  <label for="a1">Can log in to the P-Cluster</label>
</div>
<div>
  <input type="checkbox" name="a2">
  <label for="a2">Update the .bashrc file in your home directory on the P-Cluster</label>
</div>
```
The P-Cluster is very much like a high-performance computing (HPC) system where you can run interactive jobs and/or perform parallel computing. Our [P-Cluster](https://docs.aws.amazon.com/parallelcluster/latest/ug/what-is-aws-parallelcluster.html) uses compute resources from the AWS cloud.




## Connecting to the P-Cluster

### Generating SSH keys

You will need to use ssh keys to log into the P-Cluster, there are no passwords.
To generate ssh keys, use a tool like [ssh-keygen](https://linux.die.net/man/1/ssh-keygen).
    
```
ssh-keygen -q -N "" -C "your_email@address.edu" -t rsa -b 4096 -f PATH_AND_FILENAME_OF_NEW_KEY
```

The ```-q and -N ""``` arguments means stay quiet and do not use a passphrase (optional).

Your ```PATH_AND_FILENAME_OF_NEW_KEY`` should be something like the following:

```
ssh-keygen -q -N "" -C "your_email@address.edu" -t rsa -b 4096  -f ~/.ssh/id_rsa_key_PCLUSTER 
```
    
Now you will have a pair of public and private keys at the specified location. The public key has the extension ```.pub``` and the private key has the same filename with no extension. So your PUBLIC KEY will be something like ```id_rsa_key_PCLUSTER.pub``` and your PRIVATE KEY will be something like ```id_rsa_key_PCLUSTER```. These files will be together in your key directory (e.g., ~/.ssh).

```{attention}
Before you can connect to the P-Cluster with your private key, you must send your *public* SSH key to [eccohackweek@gmail.com](mailto:eccohackweek@gmail.com) so your public key can be registered. Do not send your *private* key. 
```

### Connecting to the P-Cluster using SSH and your private key

From your local Linux or Mac terminal, connect to the P-Cluster via ssh using its IP address (34.210.1.198):
```
ssh -i /path/to/privatekey -X USERNAME@34.210.1.198
```

Note your USERNAME is not the same as your email address, it will be emailed to you.

The -X option in the ssh -X command enables X11 forwarding, allowing you to run graphical applications on a remote machine and display them locally on your computer.

If you used a passphrase to create your keys then you may be prompted to enter it:
```    
    Enter passphrase for key '/Users/USERNAME/.ssh/id_rsa':
```


## P-Cluster Overview
Once you successfully log in, you will be in your home directory, /home/USERNAME/. The home directory is intended for storing analysis code, scripts, configurations, and user-specific settings, while the /efs/USERNAME/ directory is better suited for conducting model runs.


### Shell Environment 
By default, the P-Cluster uses Bash as the shell environment. Each user has a .bashrc file in their home directory, which is automatically executed whenever a new Bash session starts. This file can be customized to set environment variables, create aliases, and configure other shell settings. Here is an [example .bashrc](../tutorials/pcluster/example.bashrc) that sets the default Bash environment on the P-Cluster. This [example .bashrc](../tutorials/pcluster/example.bashrc) can serve as a starting point for further customization (download it, place it in your home directory, and rename it to .bashrc).

If preferred, users can switch to other shells such as Csh, Ksh, and Zsh. 

### Spack
The P-Cluster uses Spack (https://spack.io), an open-source package manager, to manage modules and libraries. The [example .bashrc](../tutorials/pcluster/example.bashrc) initializes Spack profile. One can use Spack to install software and generate new modules even as a non-root user, although the collection of modules on the P-Cluster is already extensive.  

### The `module` Command

The `module` command is used on the P-Cluster to manage environment modules. Modules allow users to easily load, unload, and switch between different software environments without manually modifying environment variables like `PATH` and `LD_LIBRARY_PATH` manually. This command is especially useful for managing multiple versions of software or libraries in shared environments.

When you load a module, it configures your environment to use a specific version of software. You can list available modules, load and unload modules, and reset your environment using various `module` subcommands.

Below is a list of common `module` commands and their functions:

| Command             | Description                                                                            | Example                    |
|---------------------|----------------------------------------------------------------------------------------|----------------------------|
| `module avail`      | Lists all available modules that can be loaded.                                         | `module avail`             |
| `module list`       | Shows a list of currently loaded modules in your environment.                           | `module list`              |
| `module load`       | Loads a specific module into your environment, making the software available for use.   | `module load gcc/9.3.0`    |
| `module unload`     | Unloads a specific module, removing it from your environment.                           | `module unload gcc/9.3.0`  |
| `module purge`      | Unloads all currently loaded modules, resetting your environment.                       | `module purge`             |

### Batch System
The P-Cluster uses Slurm (https://slurm.schedmd.com) as the batch system for submitting jobs for MPI runs. Some commonly used commands are listed below.

| Command   | Description                                                                 | Example                                |
|-----------|-----------------------------------------------------------------------------|----------------------------------------|
| `sbatch`  | Submits a job script to the Slurm scheduler.                                | `sbatch job_script.sh`                 |
| `scancel` | Cancels a pending or running job.                                           | `scancel <job_id>`                     |
| `squeue`  | Displays information about jobs in the queue.                              | `squeue`                                |
| `sinfo`   | Displays information about available Slurm nodes and partitions.            | `sinfo`                                |
| `salloc`  | Allocates resources for a job interactively.                               | `salloc --ntasks=2 --ntasks-per-node=2 --partition=sealevel-c5xl-demand --time=01:00:00`    |
| `srun`    | Submits a job or launches parallel tasks (can be used in a script or interactively). | `srun --ntasks=4 ./my_program`|

There are numerous resources on the web about how to use Slurm. One useful example can be found [here](https://hpc.nmsu.edu/discovery/slurm/commands/#_slurm_script_main_parts).

Note that the head node (the machine where you first log in to 34.210.1.198) has very limited resources and is suitable for editing, but not for intensive data analysis or processing. We recommend using salloc to request an interactive node for heavy data processing. For instance, issue the following command:
```
salloc --ntasks=2 --ntasks-per-node=2 --partition=sealevel-c5xl-demand --time=01:00:00 
```
This command requests an interactive node on the `partition` `sealevel-c5xl-demand` with two tasks (a term similar to processes) running on it for one hour. Note that the term `partition` is analogous to `queue` in [Portable Batch System](https://altair.com/pbs-professional), a.k.a, `PBS`, another commonly used batch system that users may be familiar with. After issuing the `salloc` command, one would receive the following message on the screen with a job identification number (ID), as shown below (using 123 as an example ID):
```
salloc: Granted job allocation 123
```
**`Slurm` may take several minutes to allocate and configure the requested resources.** Once the resources are ready, the prompt will appear as follows:
```
USERANME@ip-10-20-22-69:/efs/USERNAME$ 
```
Then, one can run command or executable scripts. If the interactive node is no longer needed, use `scancel JOB_ID` to exit the partition and return the requested resources.

In addition to `salloc`, `srun` can also be used to request an interactive node, though it is often used to run a specific script or job. `salloc`, on the other hand, allows users to run multiple commands once the resources are allocated.